In [5]:
#Test the pairs of sequences using Smith–Waterman algorithm :
    •GATTACA
    •ATTACATTAC
    
    •ATATATATATA
    •ATATATATAT
    
    •AATAATAATAAT
    •AAAATAAATAAA
    
    •ATATACACACA
    •ATATGTATACAT


#function to read a substitution matrix from file
#where the scores are separated by tabs and the first row is used to define the alphabet used
def read_submat_file(filename): 
    sm = { }
    f = open(filename, "r")
    line = f.readline()# returns one line from the file
    line  =line.rstrip()#removes any trailing characters like space
    tokens = line.split() # splits a string into a list 
    ns = len(tokens)
    #print(ns)
    alphabet = []
    for i in range(0, ns):
        alphabet.append(tokens[i][0]) 
    for i in range(0,ns):
        line = f.readline()
        line = line.rstrip()
        line = line.rstrip('\n')
        tokens = line.split()

        for j in range(1, len(tokens)-1):
            k = alphabet[i]+alphabet[j]
            sm[k] = int(tokens[j])
    return sm

In [6]:
#function to show how the score of an alignment,represented by a list with two sequences,containing gaps, can be caculated
def score_pos (c1, c2, sm, g): #
    if c1 == "−" or c2=="−": 
        return g
    else:
        return sm[c1+c2]

#function to bulid the S and T matrices,also returning the maximun score
def smith_Waterman (seq1, seq2, sm, g): 
    S = [[0]]
    T = [[0]]
    maxscore = 0
    for j in range(1, len(seq2)+1):
 
        S[0].append(0) #adds an item to the end of the list.
        T[0].append(0) 
    for i in range(1, len(seq1)+1):
        S.append([0])
        T.append([0])
    for i in range(0, len(seq1)):
        for j in range(len(seq2)):
            s1 = S[i][j] + score_pos (seq1[i], seq2[j], sm, g); 
            s2 = S[i][j+1] + g
            s3 = S[i+1][j] + g
            b = max(s1, s2, s3)
            if b <= 0: 
                S[i+1].append(0) 
                T[i+1].append(0)
            else:
                S[i+1].append(b) 
                T[i+1].append(max3t(s1, s2, s3))
                if b > maxscore:
                    maxscore = b
    return (S, T, maxscore)

In [7]:
#funciton to handle one optimal alignment 
def recover_align_local (S, T, seq1, seq2): 
    res = ["", ""]
    i, j = max_mat(S) 
    while T[i][j]>0:
        if T[i][j]==1:
            res[0] = seq1[i-1] + res[0] 
            res[1] = seq2[j-1] + res[1]
            i -= 1
            j -= 1
        elif T[i][j] == 3:
            res [0] = "-" + res [0];
            res[1] = seq2[j-1] + res[1]
            j -= 1
        
        elif T[i][j] == 2:
            res[0] = seq1[i-1] + res[0]
            res [1] = "−" + res [1]
            i -= 1
    return res
def max_mat(mat):
    maxval = mat [0][0]
    maxrow = 0
    maxcol = 0
    for i in range(0,len(mat)):
        for j in range(0, len(mat[i])):
            if mat[i][j] > maxval:
                maxval = mat[i][j]
                maxrow = i
                maxcol = j
    return (maxrow,maxcol)

def max3t (v1, v2, v3):
    if v1 > v2:
        if v1 > v3: 
            return 1
        else: 
            return 3
    else:
        if v2 > v3: return 2
        else: return 3
    

In [25]:
def print_mat2(mat,seq1,seq2):
    print('    ','',end='')
    for  s1 in seq2:
        print(s1,'',end='')
    print('\n','',end='')
    print(' ','',end='')
    for s_ in mat[0]:
        print(s_,'',end='')
    print('\n','',end='')
    for i in range(1, len(mat)): 
        m = [str(m1) for m1 in mat[i] ]
        s = [seq1[i-1]] +m 
        for s_ in s:
            print(s_, '',end='')
        print('\n','',end='')
    
def test_local_alig2():
    #sm = read_submat_file("blosum62.txt")
    sm = create_submat(1,0,"ACGT")
    #print(sm)
    seq1 = "GATTACA"
    seq2 = "ATTACATTAC"
    res = smith_Waterman(seq1, seq2,sm, 0)
    S = res[0]
    T = res[1]
    #print(res)
    print("Score of optimal alignment:", res[2])
    print("Score matrix is")
    print_mat2(S,seq1,seq2)
    print("Trace-back matrix is")
    print_mat2(T,seq1,seq2)
    #print_mat2(T,seq1,seq2)
    alinL= recover_align_local(S, T, seq1, seq2)
    print(alinL[0])
    
test_local_alig2()
from Bio import pairwise2
from Bio.pairwise2 import format_alignment
alignments = pairwise2.align.globalxx("GATTACA", "ATTACATTAC")
print (len(alignments))
for a in alignments: 
    print(format_alignment(*a))

# Trace-back matrix 1—Diagonal，2—vertical ，3-horizontal


Score of optimal alignment: 6
Score matrix is
     A T T A C A T T A C 
   0 0 0 0 0 0 0 0 0 0 0 
 G 0 0 0 0 0 0 0 0 0 0 0 
 A 0 1 1 1 1 1 1 1 1 1 1 
 T 0 1 2 2 2 2 2 2 2 2 2 
 T 0 1 2 3 3 3 3 3 3 3 3 
 A 0 1 2 3 4 4 4 4 4 4 4 
 C 0 1 2 3 4 5 5 5 5 5 5 
 A 0 1 2 3 4 5 6 6 6 6 6 
 Trace-back matrix is
     A T T A C A T T A C 
   0 0 0 0 0 0 0 0 0 0 0 
 G 0 0 0 0 0 0 0 0 0 0 0 
 A 0 1 3 3 3 3 3 3 3 3 3 
 T 0 2 1 3 3 3 3 3 3 3 3 
 T 0 2 2 1 3 3 3 3 3 3 3 
 A 0 2 2 2 1 3 3 3 3 3 3 
 C 0 2 2 2 2 1 3 3 3 3 3 
 A 0 2 2 2 2 2 1 3 3 3 3 
 ATTACA
2
GATTACA----
 ||||||    
-ATTACATTAC
  Score=6

GATTAC---A-
 |||||   | 
-ATTACATTAC
  Score=6



In [26]:
def print_mat2(mat,seq1,seq2):
    print('    ','',end='')
    for  s1 in seq2:
        print(s1,'',end='')
    print('\n','',end='')
    print(' ','',end='')
    for s_ in mat[0]:
        print(s_,'',end='')
    print('\n','',end='')
    for i in range(1, len(mat)): 
        m = [str(m1) for m1 in mat[i] ]
        s = [seq1[i-1]] +m 
        for s_ in s:
            print(s_, '',end='')
        print('\n','',end='')
    
def test_local_alig2():
    #sm = read_submat_file("blosum62.txt")
    sm = create_submat(1,0,"ACGT")
    #print(sm)
    seq1 = "ATATATATATA"
    seq2 = "ATATATATAT"
    res = smith_Waterman(seq1, seq2,sm, 0)
    S = res[0]
    T = res[1]
    #print(res)
    print("Score of optimal alignment:", res[2])
    print("Score matrix is")
    print_mat2(S,seq1,seq2)
    print("Trace-back matrix is")
    print_mat2(T,seq1,seq2)
    #print_mat2(T,seq1,seq2)
    alinL= recover_align_local(S, T, seq1, seq2)
    print(alinL[0])
    
test_local_alig2()
from Bio import pairwise2
from Bio.pairwise2 import format_alignment
alignments = pairwise2.align.globalxx("ATATATATATA", "ATATATATAT")
print (len(alignments))
for a in alignments: 
    print(format_alignment(*a))


Score of optimal alignment: 10
Score matrix is
     A T A T A T A T A T 
   0 0 0 0 0 0 0 0 0 0 0 
 A 0 1 1 1 1 1 1 1 1 1 1 
 T 0 1 2 2 2 2 2 2 2 2 2 
 A 0 1 2 3 3 3 3 3 3 3 3 
 T 0 1 2 3 4 4 4 4 4 4 4 
 A 0 1 2 3 4 5 5 5 5 5 5 
 T 0 1 2 3 4 5 6 6 6 6 6 
 A 0 1 2 3 4 5 6 7 7 7 7 
 T 0 1 2 3 4 5 6 7 8 8 8 
 A 0 1 2 3 4 5 6 7 8 9 9 
 T 0 1 2 3 4 5 6 7 8 9 10 
 A 0 1 2 3 4 5 6 7 8 9 10 
 Trace-back matrix is
     A T A T A T A T A T 
   0 0 0 0 0 0 0 0 0 0 0 
 A 0 1 3 3 3 3 3 3 3 3 3 
 T 0 2 1 3 3 3 3 3 3 3 3 
 A 0 2 2 1 3 3 3 3 3 3 3 
 T 0 2 2 2 1 3 3 3 3 3 3 
 A 0 2 2 2 2 1 3 3 3 3 3 
 T 0 2 2 2 2 2 1 3 3 3 3 
 A 0 2 2 2 2 2 2 1 3 3 3 
 T 0 2 2 2 2 2 2 2 1 3 3 
 A 0 2 2 2 2 2 2 2 2 1 3 
 T 0 2 2 2 2 2 2 2 2 2 1 
 A 0 2 2 2 2 2 2 2 2 2 2 
 ATATATATAT
1
ATATATATATA
|||||||||| 
ATATATATAT-
  Score=10

